# Validation of Two Earth-mass planets orbiting GJ 1002

*Paula Andrea Castro Nieva* 

This Jupyter Notebook serves to accompany the Research Note of the AAS with the purpose of documenting the experiments shown in the publication.  


## Imports 

First we have to install the NWelch package by pip installing:

In [ ]:
pip install -i https://test.pypi.org/simple/NWelch

The following code block imports all the other necessary packages. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.timeseries import LombScargle

from NWelch import TimeSeries
from NWelch import Bivariate as Bi

## Read the Data 

Data are from <a href="https://ui.adsabs.harvard.edu/abs/2023A%26A...670A...5S/abstract">	
Mascareño et al. (2023)</a>

In [ ]:
data = pd.read_csv('GJ1002_SuarezMascareno_2023.txt', delim_whitespace=True,
                  header=None, comment='#', skiprows=50,
                  names=['BJD-2450000', 'RV1', 'RV2', 'RV3', 'RV4', 'e_RV', 'FWHM1', 'FWHM2',
                         'FWHM3', ' e_FWHM', 'Temp', 'Inst'])
data  

In [ ]:
file = 'GJ1002_SuarezMascareno_2023.txt'
time, rvobs, rverr, fwhm,fwhmerr, number = np.loadtxt(file, usecols=[0, 2, 5, 8,9,11], unpack=True, 
                                      skiprows=50, comments='#')


In [ ]:
colormap = {0: 'blue',  # CARMENES
            1: 'red',   # ESPRESSO 18
            2: 'green', # ESPRESSO 19
            3: 'orange'} # ESPRESSO 21

colors = [colormap[int(n)] for n in number]

In [ ]:
rv = TimeSeries.TimeSeries(time,rvobs)
scatter = plt.scatter(time - time[0], rvobs, c=colors, label='Data with error', cmap='Set1')
for i in range(len(time)):
    plt.errorbar(time[i] - time[0], rvobs[i], yerr=rverr[i], fmt='o', color=colors[i])
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=8, label=label) 
           for label, col in zip(['CARMENES', 'ESPRESSO 18', 'ESPRESSO 19', 'ESPRESSO 21'], 
                                 ['blue', 'red', 'green', 'orange'])]
plt.legend(handles=handles,fontsize='small')
plt.xlabel('BJD-2450000')
plt.ylabel('RV (m/s)')
plt.title('GJ 1002')
print('Number of observations:', rv.N)

In [ ]:
FWHM = TimeSeries.TimeSeries(time,fwhm)
scatter = plt.scatter(time - time[0], fwhm, c=colors, label='Datos con error', cmap='Set1')
for i in range(len(time)):
    plt.errorbar(time[i] - time[0], fwhm[i], yerr=fwhmerr[i], fmt='o', color=colors[i])
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=8, label=label) 
           for label, col in zip(['CARMENES', 'ESPRESSO 18', 'ESPRESSO 19', 'ESPRESSO 21'], 
                                 ['blue', 'red', 'green', 'orange'])]
plt.legend(handles=handles,fontsize='small')
plt.xlabel('BJD-2450000')
plt.ylabel('FWHM ')
plt.title('GJ 1002')
print('Number of observations:', FWHM.N)

## NWelch Analysis



In [ ]:
nyquistm = 0.03504
rv.segment_data(2, 5*nyquistm, window = 'None', plot_windows = True)
rv.Welch_powspec()
rv.Welch_powspec_bootstrap()


FWHM.segment_data(2, 5*nyquistm, window = 'None', plot_windows = True)
FWHM.Welch_powspec()
FWHM.Welch_powspec_bootstrap()

In [ ]:
pb = 10.3465 # Period planet b report in the papper
pc =  20.202 # Period planet c report in the papper
prot = 126 # Period rotation report in the papper
planets = [1/pb, 1/pc]

rv.powplot(Welch = True,title='TERRA RV ', vlines=planets )
plt.xlabel('Frequency (1/day)')
plt.ylabel(r"$\hat{S}_{xx}(f)$")
plt.ylim(10**-1,10**2)
plt.text(0.092,0.15,r"b")
plt.text(0.050,0.15,r"c")
plt.tight_layout()
plt.savefig('TERRARVNWelch.pdf',format='pdf')


FWHM.powplot(Welch = True,title='Temperature-corrected FWHM ', vlines=[1/pc, 0.5*1/pc])
plt.xlabel('Frequency (1/day)')
plt.ylabel(r"$\hat{S}_{xx}(f)$")
plt.text(0.025,1,r"$\frac{f_c}{2}$")
plt.text(0.050,1,r"c")
plt.tight_layout()
plt.savefig('FWHMNWelch.pdf',format='pdf')

## Pseudowindow
Create four versions of a fake signal moving the phase with the same period as each planet.
Calculate and plot the Welch's power spectrum and spectral window

planet b

In [ ]:
t = time
RV1F=np.zeros((4,len(t)))

phase=[0,np.pi/2,np.pi,3*np.pi/4]

for j in range(len(phase)):
    RV1F[j,:]=np.sin(2 * np.pi * 1/pb * t + phase[j])

In [ ]:
RV1FT =[]

for k in range(4):
    RV1FT.append(TimeSeries.TimeSeries(t,RV1F[k,:]))

In [ ]:
nyquistm = 0.03504
for u in range(4):
    RV1FT[u].segment_data(2, 5*nyquistm, window = 'None', plot_windows = True)
    RV1FT[u].Welch_powspec()
    RV1FT[u].Welch_powspec_bootstrap()

In [ ]:
for w in range(4):
    RV1FT[w].powplot(title= 'Planet b pseudowindow', Welch = True, vlines=planets)
    plt.legend(['Planet b ' f' Phase = {phase[w]:.2f}'], loc='upper right')
    plt.xlabel('Frequency (1/day)')
    plt.ylabel('W(f)')
    plt.ylim(10**-2,20)
    plt.xlim(0,0.180)
    plt.text(0.092,0.05,r"b")
    plt.text(0.050,8,r"c")
    if w==0: 
        plt.tight_layout()
        plt.savefig('pseudowindowb.pdf',format='pdf')


planet c

In [ ]:
RV2F=np.zeros((4,len(t)))

phase=[0,np.pi/2,np.pi,3*np.pi/4]

for j in range(len(phase)):
    RV2F[j,:]=np.sin(2 * np.pi * 1/pc * t + phase[j])

In [ ]:
RV2FT =[]

for k in range(4):
    RV2FT.append(TimeSeries.TimeSeries(t,RV2F[k,:]))

In [ ]:
nyquistm = 0.03504
for u in range(4):
    RV2FT[u].segment_data(2, 5*nyquistm, window = 'None', plot_windows = True)
    RV2FT[u].Welch_powspec()
    RV2FT[u].Welch_powspec_bootstrap()

In [ ]:
for w in range(4):
    RV2FT[w].powplot(title= 'Planet c pseudowindow', Welch = True, vlines=planets)
    plt.legend(['Planet c' f' Phase = {phase[w]:.2f}'])
    plt.xlabel('Frequency (1/day)')
    plt.ylabel('W(f)')
    plt.ylim(10**-2,30)
    plt.text(0.092,10,r"b")
    plt.text(0.050,0.05,r"c")

## Siegels Tests

In [ ]:
rv.Siegel_test(Welch=True)
FWHM.Siegel_test(Welch=True)
rv.Siegel_test(Welch=True,tri=True)
FWHM.Siegel_test(Welch=True,tri=True)

## Frequency of highest periodogram peak

In [ ]:
N1 = np.random.randn(len(time))
N2=np.random.randn(len(rvobs))
N3=np.random.randn(len(fwhm))

In [ ]:
RV2=rvobs+(N2*rverr)
fwhmN=fwhm+(N3*fwhmerr)

plot the time series to make sure it looks similar to the original RVs  

In [ ]:
rv_ts2 = TimeSeries.TimeSeries(time,RV2)
rv_ts2.scatterplot( xlabel='Time (days)', ylabel='RV (m/s)' )
plt.errorbar(time-time[0], RV2, yerr=rverr, fmt='o', color='darkorchid', label='Datos wirh error')
print('Number of observations:', rv_ts2.N)


Repeat the same procedure 10000 times and calculated the highest peak in the periodograms

In [ ]:
num_iterations = 10000
highest_peak_frequencies = []

for _ in range(num_iterations):
    N2 = np.random.randn(len(rvobs))

    RV2 = rvobs + (N2 * rverr)

    rv_ts=TimeSeries.TimeSeries(time,RV2)
    rv_ts.frequency_grid(0.15982)
    rv_ts.pow_FT(N_bootstrap=10)
    
    
    highest_peak_freq = rv_ts.powfgrid[np.argmax(rv_ts.power)]
    highest_peak_frequencies.append(highest_peak_freq)

Plot the distribution of the frequencies 

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(highest_peak_frequencies, bins=50, color='blue', edgecolor='black')
plt.xlabel('Frequency (1/days)')
plt.ylabel('Iterations')
plt.title('Monte Carlo period search')
plt.text(0.094,3000,r"b")
plt.text(0.045,1500,r"c")
plt.grid(color='0.85')
plt.tight_layout()
plt.savefig('MontecarloHistrogram.pdf',format='pdf')
plt.show()
